In [445]:
input_file = "input.txt"

In [446]:
blocks = open(input_file, 'r').read().split('\n\n')

In [447]:
wflows, parts = [block.split('\n') for block in blocks]
wflows = [wflow.split('{') for wflow in wflows]
wflows = [(wflow[0], wflow[1][:-1].split(',')) for wflow in wflows]
wflows = [ (label, [tuple(step.split(':') if ':' in step else ['True', step]) for step in flow]) for label, flow in wflows]
parts = parts[:-1]
parts = [part[1:-1].split(',') for part in parts]
parts = [[x.split('=') for x in part] for part in parts]
parts = [{x[0]: int(x[1]) for x in part} for part in parts]

#Part 1

In [449]:
def wflowFactory(wflow):
    def f(part):
        x = part['x']
        m = part['m']
        a = part['a']
        s = part['s']
        for step in wflow[1]:
            if eval(step[0].replace('x', 'int(x)').replace('m', 'int(m)').replace('a', 'int(a)').replace('s', 'int(s)')):
                if step[1] == 'A':
                    return True
                if step[1] == 'R':
                    return False
                return (workflows[step[1]](part))
    return f

In [450]:
workflows = dict()
for wflow in wflows:
    workflows[wflow[0]] = wflowFactory(wflow)

In [451]:
def is_accepted(part):
    return workflows['in'](part)

In [452]:
part1 = 0
for part in parts:
    if is_accepted(part):
        for k in part:
            part1 += part[k]

In [453]:
part1

432427

# Part 2

In [ ]:
from collections import defaultdict

In [ ]:
def constant_factory(value):
    return lambda: set({1,4000})

bounds = defaultdict(constant_factory('<missing>'))
for flow in wflows:
    for rule in flow[1]:
        if rule[0] != 'True':
            rule = rule[0].replace('<',':').replace('>',':')
            label, bound = rule.split(':')
            bound = int(bound)
            bounds[label].add(bound-1)
            bounds[label].add(bound)
            bounds[label].add(bound+1)
        

In [ ]:
sorted_bounds = {}
for k in bounds:
    sorted_bounds[k] = sorted(list(bounds[k]))


In [ ]:
weigthed_bounds = {}
for k in bounds:
    weigthed_bounds[k] = [(sorted_bounds[k][ix], sorted_bounds[k][ix+1]-sorted_bounds[k][ix]) for ix, _ in enumerate(sorted_bounds[k][:-1])]

In [ ]:
for r in "xmas":
    weigthed_bounds[r].append((4000, 1))
weigthed_bounds

In [ ]:
part2 = 0
for x_bound, x_weight in weigthed_bounds['x']:
    for m_bound, m_weight in weigthed_bounds['m']:
        for a_bound, a_weight in weigthed_bounds['a']:
            for s_bound, s_weight in weigthed_bounds['s']:
                part = {'x':x_bound,'m':m_bound,'a':a_bound,'s':s_bound}
                weight = x_weight*m_weight*a_weight*s_weight
                if is_accepted(part):
                    part2 += weight

#does not scale on input.txt

In [ ]:
part2

# Part 2 - alternative

In [454]:
all_parts = {'x': [1,4000], 'm': [1,4000], 'a': [1,4000], 's': [1,4000]}

In [456]:
from functools import reduce


In [457]:
def cut_itv(itv, sign, bound):
    """
    itv should be [10,678] like
    rule should be x>400 like
    """

    if '<':
        accepted_itv = [itv[0],min(bound-1,itv[1])]
        to_process_itv = [max(bound, itv[0]), itv[1]]
    if sign == '>':
        accepted_itv = [max(bound+1,itv[0]), itv[1]]
        to_process_itv = [itv[0],min(bound, itv[1])]

    if accepted_itv[1] < accepted_itv[0]:
        accepted_itv = []
    if itv[1] < itv[0]:
        to_process_itv = []
    
    return accepted_itv, to_process_itv

In [459]:
def cardinality(part_itvs):
    if any(part_itvs[k] == [] for k in part_itvs):
        return 0
    return reduce(lambda a,b:a*b, [1+part_itvs[k][1]-part_itvs[k][0] for k in part_itvs])

In [461]:
def wflowItvFactory(wflow):
    def f(part_itvs):
        to_process_itvs = dict(part_itvs)
        accepted_itvs = dict(part_itvs)

        res = 0
        for step in wflow[1]:
            accepted_itvs = dict(to_process_itvs)
            if '<' in step[0] or '>' in step[0]:
                if '<' in step[0]:
                    sym, bound = step[0].split('<')
                    accepted, to_process = cut_itv(to_process_itvs[sym], '<', int(bound))
                if '>' in step[0]:
                    sym, bound = step[0].split('>')
                    accepted, to_process = cut_itv(to_process_itvs[sym], '>', int(bound))

                to_process_itvs[sym] = to_process
                accepted_itvs[sym] = accepted
            
            if step[0] == 'True':
                to_process_itvs = {'x':[], 'm':[], 'a':[], 's':[]}
            


            if step[1] == 'R':
                res += 0
            elif step[1] == 'A':
                res += cardinality (accepted_itvs)
            else:
                res += workflow_itvs[step[1]](accepted_itvs)

            if any(to_process_itvs[k] == [] for k in to_process_itvs):
                break

        return res
    return f

In [462]:
workflow_itvs = dict()
for wflow in wflows:
    workflow_itvs[wflow[0]] = wflowItvFactory(wflow)

In [464]:
part2 = workflow_itvs['in'](all_parts)
part2

143760172569135